In [10]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from lightning import Fabric # Needs GPU!

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.01
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [11]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = ConvNeuralNet(num_classes)
# Define the model from the class above

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer, here SGD is selected.
# Play around with weight_decay and momentum if needed. 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.010, momentum = 0.9)  

total_step = len(train_loader)

In [6]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        
        # Move tensors to the configured device
        
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Note that we need to clear the gradients of all variables: 
        # In PyTorch, for every mini-batch during the training phase, we typically want to 
        # explicitly set the gradients to zero before starting to do backpropagation 
        # (i.e., updating the Weights and biases) because PyTorch accumulates the gradients 
        # on subsequent backward passes. This accumulating behavior is convenient 
        # while training RNNs or when we want to compute the gradient of the loss summed 
        # over multiple mini-batches. So, the default action has been set to accumulate (i.e. sum) 
        # the gradients on every loss.backward() call

        # Backward and optimize
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

            

Epoch [1/10], Loss: 1.5737
Epoch [2/10], Loss: 1.8112
Epoch [3/10], Loss: 1.2873
Epoch [4/10], Loss: 1.5460
Epoch [5/10], Loss: 1.5680
Epoch [6/10], Loss: 1.1948
Epoch [7/10], Loss: 1.1160
Epoch [8/10], Loss: 0.5954
Epoch [9/10], Loss: 0.7359
Epoch [10/10], Loss: 1.2340


We now run the model in training data. We do not do gradient since we 
do not want to update weights. 

In [8]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))



Accuracy of the network on the 50000 train images: 72.322 %
